### NOMIS API Data Extract for Historic Census Data.

The NOMIS RESTful API limits the size of the 'queries / calls' you can request.
For example, you cannot request all of the geography codes in a single call, the site will time-out or error due to the number of data items. Additionally each call is limited to a maximum of 25k records; hence the need for a few nested programming loops etc. etc. etc.

***Here are is a useful reading resource:***
https://www.nomisweb.co.uk/api/v01/help

***Here is the target data Darren Barnes has requested:***

"*Had a chat with census team this morning and this is the dataset they would like us to test out
Its a 2011 census table LC3409EW >> https://www.nomisweb.co.uk/census/2011/lc3409ew and includes General Health, Tenure and Age, covers England and Wales, and goes down to OA level.
obviously its available through the NOMIS API and i would like us to suck that into COGS rather than download and transform. The census team have had some very good feedback from Ahmed on the data viz and sparql and i think this has generated some buzz within the census senior management team. So the idea is to load this data into COGS and have Ahmed do a data viz on it and compare that with the CMD.
We have looked at NOMIS API already and i would like us to give consideration on what and when we could get this loaded into COGS."*

**OA**: Census **O**utput **A**reas.

In [ ]:
# Components & Libraries:
import pandas as pd
import json
import slug
from pathlib import Path
import datetime
import numpy as np

from cachecontrol import CacheControl
from cachecontrol.caches import FileCache
from cachecontrol.heuristics import ExpiresAfter
from requests import Session
session = CacheControl(Session(), cache=FileCache('.cache'), heuristic=ExpiresAfter(days=7))

import re

pd.set_option('display.max_colwidth', -1)

out = Path('out')
out.mkdir(exist_ok=True)

absolute_start = datetime.datetime.now()

In [ ]:
# NOMIS RESTful: Get ID for Dataset from NOMIS Data Families:

# Get Family List:
baseURL_ID = 'https://www.nomisweb.co.uk/api/v01/dataset/def.sdmx.json'
df_ID = pd.read_json(baseURL_ID)
display(df_ID)

i = 0
while True:
    str_df_ID = str(df_ID.structure[2]['keyfamily'][i]['name']['value'])
    if 'General health by tenure by age' in str_df_ID:
        print(str_df_ID + ' -- ID: ' + str(df_ID.structure[2]['keyfamily'][i]['id']))
        str_dataset_id = str(df_ID.structure[2]['keyfamily'][i]['id'])
        break
    i += 1

df_ID = pd.DataFrame() # Memory


In [ ]:
# NOMIS RESTful: Get Parent Level Geography for Dataset:

# Get Geography sets using our ID:
baseURL_Geography = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '/geography.def.sdmx.json'
df_Geography = pd.read_json(baseURL_Geography)
display(df_Geography)

i = 0
flag_england = False
flag_wales = False
while True:
    str_df_Geog_ID = str(df_Geography.structure[0]['codelist'][0]['code'][i]['description']['value'])
    if (str_df_Geog_ID == 'England'):
        print(str_df_Geog_ID + ' -- ID: ' + str(df_Geography.structure[0]['codelist'][0]['code'][i]['value']))
        str_df_Geog_ID_England = str(df_Geography.structure[0]['codelist'][0]['code'][i]['value'])
        flag_england = True
    if (str_df_Geog_ID == 'Wales'):
        print(str_df_Geog_ID + ' -- ID: ' + str(df_Geography.structure[0]['codelist'][0]['code'][i]['value']))
        str_df_Geog_ID_Wales = str(df_Geography.structure[0]['codelist'][0]['code'][i]['value'])
        flag_wales = True
    if flag_england == True and flag_wales == True:
        break
    i += 1

df_Geography = pd.DataFrame() # Memory
    

In [ ]:
# NOMIS RESTful: Get Hierarchical Geography for Dataset using Parent Geography IDs:

# Get Geography sets using Parent Level Geography IDs for WALES:
baseURL_Geography_Wales = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '/geography/' + str_df_Geog_ID_Wales + '.def.sdmx.json'
df_Geography_Wales = pd.read_json(baseURL_Geography_Wales)
display(df_Geography_Wales)

i = 0
while True:
    str_df_Geog_L1_ID = str(df_Geography_Wales.structure[0]['codelist'][0]['code'][i]['description']['value'])
    if '2011 output areas within Wales' == str_df_Geog_L1_ID:
        print(str_df_Geog_L1_ID + ' -- ID: ' + str(df_Geography_Wales.structure[0]['codelist'][0]['code'][i]['value']))
        str_df_Geog_L1_ID_Wales = str(df_Geography_Wales.structure[0]['codelist'][0]['code'][i]['value'])
        break
    i += 1
    
df_Geography_Wales = pd.DataFrame() # Memory


In [ ]:
# NOMIS RESTful: Get Hierarchical Geography for Dataset using Parent Geography IDs:

# Get Geography sets using Parent Level Geography IDs for England:
baseURL_Geography_England = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '/geography/' + str_df_Geog_ID_England + '.def.sdmx.json'
df_Geography_England = pd.read_json(baseURL_Geography_England)
display(df_Geography_England)

i = 0
while True:
    str_df_Geog_L1_ID = str(df_Geography_England.structure[0]['codelist'][0]['code'][i]['description']['value'])
    if '2011 output areas within England' == str_df_Geog_L1_ID:
        print(str_df_Geog_L1_ID + ' -- ID: ' + str(df_Geography_England.structure[0]['codelist'][0]['code'][i]['value']))
        str_df_Geog_L1_ID_England = str(df_Geography_England.structure[0]['codelist'][0]['code'][i]['value'])
        break
    i += 1
    
df_Geography_England = pd.DataFrame() # Memory


In [ ]:
# NOMIS RESTful: Get Level 2 Geography Codes for Dataset using Level 1 Geography IDs:

# Get Geography sets using Level 1 Geography IDs for WALES:
baseURL_Geography_Wales_L1 = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '/geography/' + str_df_Geog_L1_ID_Wales + '.def.sdmx.json'
df_Geography_Wales_L2 = pd.read_json(baseURL_Geography_Wales_L1)
#display(df_Geography_Wales_L2)

i = 0
str_df_Geog_L2_ID_Wales_LowerRange = 'W99999999999999999'
str_df_Geog_L2_ID_Wales_HigherRange = ''
while True:
    try:
        str_df_Geog_L2_ID = str(df_Geography_Wales_L2.structure[0]['codelist'][0]['code'][i]['description']['value'])
        if str_df_Geog_L2_ID < str_df_Geog_L2_ID_Wales_LowerRange:
            str_df_Geog_L2_ID_Wales_LowerRange = str_df_Geog_L2_ID
        if str_df_Geog_L2_ID > str_df_Geog_L2_ID_Wales_HigherRange:
            str_df_Geog_L2_ID_Wales_HigherRange = str_df_Geog_L2_ID       
        i += 1
  
    except IndexError:
        break

df_Geography_Wales_L2 = pd.DataFrame() # Memory


In [ ]:
# NOMIS RESTful: Get Level 2 Geography Codes for Dataset using Level 1 Geography IDs:

# Get Geography sets using Level 1 Geography IDs for ENGLAND:
baseURL_Geography_England_L1 = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '/geography/' + str_df_Geog_L1_ID_England + '.def.sdmx.json'
df_Geography_England_L2 = pd.read_json(baseURL_Geography_England_L1)
#display(df_Geography_England_L2)

i = 0
str_df_Geog_L2_ID_England_LowerRange = 'E99999999999999999'
str_df_Geog_L2_ID_England_HigherRange = ''
while True:
    try:
        str_df_Geog_L2_ID = str(df_Geography_England_L2.structure[0]['codelist'][0]['code'][i]['description']['value'])
        if str_df_Geog_L2_ID < str_df_Geog_L2_ID_England_LowerRange:
            str_df_Geog_L2_ID_England_LowerRange = str_df_Geog_L2_ID
        if str_df_Geog_L2_ID > str_df_Geog_L2_ID_England_HigherRange:
            str_df_Geog_L2_ID_England_HigherRange = str_df_Geog_L2_ID       
        i += 1
        
    except IndexError:
        break
    
df_Geography_England_L2 = pd.DataFrame() # Memory


In [ ]:
print(str_df_Geog_L2_ID_Wales_LowerRange)
print(str_df_Geog_L2_ID_Wales_HigherRange)
print(str_df_Geog_L2_ID_England_LowerRange)
print(str_df_Geog_L2_ID_England_HigherRange)

In [ ]:
# set explicit types to speed things up
dtype={'DATE_NAME':np.int,
        'GEOGRAPHY_CODE': str,
        'GEOGRAPHY_TYPE': str,
        'C_TENHUK11_NAME':  str,
        'C_AGE_NAME': str,
        'C_HEALTH_NAME': str,
        'OBS_VALUE':np.float,
        'RECORD_OFFSET':np.int,
        'RECORD_COUNT':np.int}

In [ ]:
# NOMIS RESTful: Get Tabular Data using Level 2 Geography Code IDs:

# Get Data for WALES:
#baseURL_Geography_Wales_Data = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '.data.csv?date=latest&geography=' + str_df_Geog_L2_ID_Wales_LowerRange + '...' + str_df_Geog_L2_ID_Wales_HigherRange + '&c_tenhuk11=2,3,5,6&c_age=1...4&c_health=1...3&measures=20100&select=date_name,geography_name,geography_code,c_tenhuk11_name,c_age_name,c_health_name,measures_name,obs_value,obs_status_name,record_offset,record_count'
# Revised:
baseURL_Geography_Wales_Data = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '.data.csv?date=latest&geography=' + str_df_Geog_L2_ID_Wales_LowerRange + '...' + str_df_Geog_L2_ID_Wales_HigherRange + '&c_tenhuk11=2,3,5,6&c_age=1...4&c_health=1...3&measures=20100&select=date_name,geography_code,geography_type,c_tenhuk11_name,c_age_name,c_health_name,obs_value,record_offset,record_count'
print(baseURL_Geography_Wales_Data)
stream = session.get(baseURL_Geography_Wales_Data, stream=True).raw
stream.decode_content = True
final_df_wales = pd.read_csv(stream)

additionalURL = ''
intRecordController = 0
    
# The links below are limited to the first 25,000 cells per call.
while True:
    start = datetime.datetime.now()
    additionalURL = '&RecordOffset=' + str(intRecordController)
    concatenatedURL = baseURL_Geography_Wales_Data + additionalURL
    stream = session.get(concatenatedURL, stream=True).raw
    stream.decode_content = True
    
    dataframe = pd.read_csv(stream, engine='c', na_filter=False, dtype=dtype)

    if (dataframe.empty):
        break
    
    #display(dataframe)
    # Additional code due to large datasets:
    # For a very basic progress monitor (no fancy code this time round):
    print("Just received record:" + (dataframe.tail(1).RECORD_OFFSET.to_string(index=False)) + ' of ' + (dataframe.tail(1).RECORD_COUNT.to_string(index=False)) + ' >>> ' + str(round((int(dataframe.tail(1).RECORD_OFFSET.to_string(index=False)) / int(dataframe.tail(1).RECORD_COUNT.to_string(index=False)) * 100),2)) + '%... in ' + str(datetime.datetime.now() - start))
    frames = [final_df_wales, dataframe]
    final_df_wales = pd.concat(frames)
    
    intRecordController = intRecordController + 25000

    
final_df_wales.GEOGRAPHY_TYPE  = final_df_wales.GEOGRAPHY_TYPE.apply(slug.slug)
final_df_wales.C_TENHUK11_NAME = final_df_wales.C_TENHUK11_NAME.apply(slug.slug)
final_df_wales.C_HEALTH_NAME   = final_df_wales.C_HEALTH_NAME.apply(slug.slug)
final_df_wales['C_AGE_NAME']   = final_df_wales['C_AGE_NAME'].replace('Age 0 to 15', 'Y0T15').replace('Age 16 to 49', 'Y16T49').replace('Age 50 to 64', 'Y50T64').replace('Age 65 and over', 'Y_GE65') 

display(final_df_wales)

final_df_wales.to_csv("temp.csv")


In [ ]:
final_df_wales.to_csv(out / 'outfinal_df_wales.csv.gz', date_format='%Y%m%d', compression="gzip")
final_df_wales = pd.DataFrame() # Memory

In [ ]:
# NOMIS RESTful: Get Tabular Data using Level 2 Geography Code IDs:

# Get Data for ENGLAND:
#baseURL_Geography_England_Data = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '.data.csv?date=latest&geography=' + str_df_Geog_L2_ID_England_LowerRange + '...' + str_df_Geog_L2_ID_England_HigherRange + '&c_tenhuk11=2,3,5,6&c_age=1...4&c_health=1...3&measures=20100&select=date_name,geography_name,geography_code,c_tenhuk11_name,c_age_name,c_health_name,measures_name,obs_value,obs_status_name,record_offset,record_count'
# Revised:
baseURL_Geography_England_Data = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '.data.csv?date=latest&geography=' + str_df_Geog_L2_ID_England_LowerRange + '...' + str_df_Geog_L2_ID_England_HigherRange + '&c_tenhuk11=2,3,5,6&c_age=1...4&c_health=1...3&measures=20100&select=date_name,geography_code,geography_type,c_tenhuk11_name,c_age_name,c_health_name,obs_value,record_offset,record_count'
stream = session.get(baseURL_Geography_England_Data, stream=True).raw
stream.decode_content = True
final_df_england = pd.read_csv(stream)

additionalURL = ''
intRecordController = 0
# The links below are limited to the first 25,000 cells per call.
while True:
    start = datetime.datetime.now()
    additionalURL = '&RecordOffset=' + str(intRecordController)
    concatenatedURL = baseURL_Geography_England_Data + additionalURL
    stream = session.get(concatenatedURL, stream=True).raw
    stream.decode_content = True
    
    dataframe = pd.read_csv(stream, engine='c', na_filter=False, dtype=dtype)
    
    #display(dataframe)
    # Additional code due to large datasets:
    # For a very basic progress monitor (no fancy code this time round):
    try:
        print("Just received record:" + (dataframe.tail(1).RECORD_OFFSET.to_string(index=False)) + ' of ' + (dataframe.tail(1).RECORD_COUNT.to_string(index=False)) + ' >>> ' + str(round((int(dataframe.tail(1).RECORD_OFFSET.to_string(index=False)) / int(dataframe.tail(1).RECORD_COUNT.to_string(index=False)) * 100),2)) + '%... in ' + str(datetime.datetime.now() - start))
    except ValueError:
        pass # avoid potential casting error on last call
    frames = [final_df_england, dataframe]
    final_df_england = pd.concat(frames)
    intRecordController = intRecordController + 25000

    
final_df_england.GEOGRAPHY_TYPE  = final_df_england.GEOGRAPHY_TYPE.apply(slug.slug)
final_df_england.C_TENHUK11_NAME = final_df_england.C_TENHUK11_NAME.apply(slug.slug)
final_df_england.C_HEALTH_NAME   = final_df_england.C_HEALTH_NAME.apply(slug.slug)
final_df_england['C_AGE_NAME']   = final_df_england['C_AGE_NAME'].replace('Age 0 to 15', 'Y0T15').replace('Age 16 to 49', 'Y16T49').replace('Age 50 to 64', 'Y50T64').replace('Age 65 and over', 'Y_GE65') 

display(final_df_england)


In [ ]:
final_df_england.to_csv(out / 'final_df_england.csv.gz', date_format='%Y%m%d', compression="gzip")
final_df_england = pd.DataFrame() # Memory

In [ ]:
#display(df_Geography_England_L2)

In [ ]:
#df_Geography_England_L2.to_csv('final_df_england_geographies.csv', date_format='%Y%m%d')

In [ ]:
print("Completion time for complete script is: ", absolute_start - datetime.datetime.now())